# SpanMarker cross-validation evaluation

Loads per-fold models trained in `training.ipynb` and performs cross-validation. Note that KFold is used without randomization (as in the training notebook).

In [ ]:
import warnings

warnings.simplefilter(action="ignore", category=FutureWarning)

import collections

import numpy as np
import pandas as pd
from customized_spanmarker_training import NoTrainPreprocTrainer
from datasets import Dataset, DatasetDict, concatenate_datasets, load_dataset
from sklearn.model_selection import KFold
from span_marker import SpanMarkerModel
from torch.optim import AdamW
from transformers import TrainingArguments, get_scheduler

Loading datasets remotely and from parquet.

In [3]:
germeval = load_dataset("gwlms/germeval2014")["train"]
germeval = germeval.select_columns(["tokens", "ner_tags"])
krp_19jhd = Dataset.from_parquet("krp_19jhd.parquet")
krp_20jhd = Dataset.from_parquet("krp_20jhd.parquet")
rrb_19jhd = Dataset.from_parquet("rrb_19jhd.parquet")
rrb_20jhd = Dataset.from_parquet("rrb_20jhd.parquet")
gszh = Dataset.from_parquet("gszh.parquet")

Mapping features to the original germeval2014 indices.

In [4]:
krp_19jhd = krp_19jhd.map(features=germeval.features)
krp_20jhd = krp_20jhd.map(features=germeval.features)
rrb_19jhd = rrb_19jhd.map(features=germeval.features)
rrb_20jhd = rrb_20jhd.map(features=germeval.features)
gszh = gszh.map(features=germeval.features)

Evaluation.

In [5]:
kf = KFold(n_splits=5)
gradient_accumulation_steps = 2
train_batch_size = 4
n_epochs = 10

metrics_overall_dict = collections.defaultdict(list)
metrics_per_ent_dict = collections.defaultdict(list)

for i, (
    (_, krp_19jhd_eval_idx),
    (_, krp_20jhd_eval_idx),
    (_, rrb_19jhd_eval_idx),
    (_, rrb_20jhd_eval_idx),
    (_, gszh_eval_idx),
) in enumerate(
    list(
        zip(
            kf.split(np.zeros(krp_19jhd.num_rows)),
            kf.split(np.zeros(krp_20jhd.num_rows)),
            kf.split(np.zeros(rrb_19jhd.num_rows)),
            kf.split(np.zeros(rrb_20jhd.num_rows)),
            kf.split(np.zeros(gszh.num_rows)),
        )
    )
):
    print(f"Fold {i}:")

    # Selecting the test subfolds
    krp_19jhd_eval = krp_19jhd.select(krp_19jhd_eval_idx)
    krp_20jhd_eval = krp_20jhd.select(krp_20jhd_eval_idx)
    rrb_19jhd_eval = rrb_19jhd.select(rrb_19jhd_eval_idx)
    rrb_20jhd_eval = rrb_20jhd.select(rrb_20jhd_eval_idx)
    gszh_eval = gszh.select(gszh_eval_idx)

    # Concatenating the test subfolds
    eval_fold = concatenate_datasets(
        [krp_19jhd_eval, krp_20jhd_eval, rrb_19jhd_eval, rrb_20jhd_eval, gszh_eval]
    )

    fold_dataset = DatasetDict(
        {
            "eval_all": eval_fold,
            "eval_krp_19jhd": krp_19jhd_eval,
            "eval_krp_20jhd": krp_20jhd_eval,
            "eval_rrb_19jhd": rrb_19jhd_eval,
            "eval_rrb_20jhd": rrb_20jhd_eval,
            "eval_gszh": gszh_eval,
        }
    )

    # Load previously saved SpanMarker fold model
    model = SpanMarkerModel.from_pretrained(
        f"models/span-marker-ktzh-stazh-cv/fold_{i}"
    )

    # Dummy definitions for optimizer, l_r_scheduler, trainer (we don't train the model here and only use the evaluate method)
    optimizer = AdamW(model.parameters(), lr=1e-05)

    l_r_scheduler = get_scheduler(
        "polynomial",
        optimizer,
        num_warmup_steps=1,
        num_training_steps=1,
        scheduler_specific_kwargs=dict(lr_end=5e-07, power=3),
    )

    trainer = NoTrainPreprocTrainer(
        model=model,
        args=TrainingArguments(output_dir="."),
        train_dataset=None,
        eval_dataset=None,
        optimizers=(optimizer, l_r_scheduler),
    )

    #
    # Evaluation
    #

    for pred_ds in [
        "eval_all",
        "eval_krp_19jhd",
        "eval_krp_20jhd",
        "eval_rrb_19jhd",
        "eval_rrb_20jhd",
        "eval_gszh",
    ]:
        metrics = trainer.evaluate(fold_dataset[pred_ds])

        # Filter metrics
        metrics_per_ent = {}
        for m, v in metrics.items():
            if any(
                m.endswith(ent_type)
                for ent_type in [
                    "PER",
                    "LOC",
                    "ORG",
                    "PERderiv",
                    "LOCderiv",
                    "ORGderiv",
                ]
            ):
                metrics_per_ent[m] = v

        # Build metrics dataframe from dict
        metrics_per_ent_df = pd.DataFrame.from_dict(metrics_per_ent).reindex(
            ["f1", "precision", "recall", "number"]
        )
        metrics_per_ent_df = metrics_per_ent_df.rename(
            columns=lambda x: x.split("_")[-1]
        )

        # Add missing metrics
        for col in ["PER", "LOC", "ORG", "PERderiv", "LOCderiv", "ORGderiv"]:
            if col not in metrics_per_ent_df.columns:
                metrics_per_ent_df[col] = 0.0

        # Reorder columns
        metrics_per_ent_df = metrics_per_ent_df[
            ["PER", "LOC", "ORG", "PERderiv", "LOCderiv", "ORGderiv"]
        ]

        # Store dataframe for aggregation
        metrics_per_ent_dict[pred_ds].append(metrics_per_ent_df)

        print(f"[[ Evaluation {pred_ds} ]]")
        display(metrics_per_ent_df.round(2))

    del trainer
    del model

Fold 0:


/Users/adrianvanderlek/anaconda3/envs/ner2/lib/python3.11/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


[[ Evaluation eval_all ]]


,PER,LOC,ORG,PERderiv,LOCderiv,ORGderiv
f1,0.97,0.95,0.93,0.0,0.86,0.73
precision,0.95,0.94,0.92,0.0,0.81,0.69
recall,0.98,0.95,0.94,0.0,0.92,0.77
number,442.00,780.00,1201.00,0.0,38.00,47.00


[[ Evaluation eval_krp_19jhd ]]


...

,PER,LOC,ORG,PERderiv,LOCderiv,ORGderiv
f1,0.92,0.94,0.94,0.0,0.55,0.73
precision,0.93,0.96,0.94,0.0,0.60,0.57
recall,0.90,0.91,0.94,0.0,0.50,1.00
number,60.00,81.00,162.00,2.0,6.00,4.00


[[ Evaluation eval_rrb_20jhd ]]


,PER,LOC,ORG,PERderiv,LOCderiv,ORGderiv
f1,0.98,0.98,0.97,0.0,1.0,1.0
precision,0.98,0.98,0.96,0.0,1.0,1.0
recall,0.98,0.98,0.98,0.0,1.0,1.0
number,53.00,129.00,159.00,0.0,4.0,10.0


[[ Evaluation eval_gszh ]]


,PER,LOC,ORG,PERderiv,LOCderiv,ORGderiv
f1,0.88,0.94,0.92,0.0,0.96,0.7
precision,0.90,0.91,0.89,0.0,0.93,0.7
recall,0.86,0.97,0.94,0.0,1.00,0.7
number,44.00,99.00,242.00,0.0,13.00,10.0


Generating aggregated metrics across folds (mean and median).

In [6]:
mean_dfs = []

for pred_ds in [
    "eval_all",
    "eval_krp_19jhd",
    "eval_krp_20jhd",
    "eval_rrb_19jhd",
    "eval_rrb_20jhd",
    "eval_gszh",
]:
    print(
        f"""-------------------
Aggregated metrics ({pred_ds})
-------------------"""
    )
    metrics_per_ent_conc = pd.concat(metrics_per_ent_dict[pred_ds])
    metrics_per_ent_conc = metrics_per_ent_conc.groupby(metrics_per_ent_conc.index)
    print("Per-ent metrics (mean)", end="")
    mean_df = (
        metrics_per_ent_conc.mean()
        .round(2)
        .reindex(["f1", "precision", "recall", "number"])
    )
    display(mean_df)
    mean_dfs.append(
        (
            pred_ds.replace("eval_", ""),
            mean_df.rename({"f1": "F1", "precision": "P", "recall": "R"}),
        )
    )
    print("Per-ent metrics (median)", end="")
    median_df = (
        metrics_per_ent_conc.median()
        .round(2)
        .reindex(["f1", "precision", "recall", "number"])
    )
    display(median_df)

-------------------
Aggregated metrics (eval_all)
-------------------
Per-ent metrics (mean)

,PER,LOC,ORG,PERderiv,LOCderiv,ORGderiv
f1,0.97,0.96,0.93,0.33,0.85,0.74
precision,0.97,0.95,0.92,0.40,0.86,0.73
recall,0.97,0.96,0.95,0.30,0.85,0.76
number,487.20,704.80,1108.20,1.20,38.40,48.20


Per-ent metrics (median)

,PER,LOC,ORG,PERderiv,LOCderiv,ORGderiv
f1,0.97,0.96,0.94,0.0,0.86,0.73
precision,0.97,0.96,0.92,0.0,0.86,0.72
recall,0.97,0.97,0.95,0.0,0.83,0.77
number,493.00,685.00,1061.00,2.0,41.00,47.00


-------------------
Aggregated metrics (eval_krp_19jhd)
-------------------
Per-ent metrics (mean)

...

,PER,LOC,ORG,PERderiv,LOCderiv,ORGderiv
f1,0.95,0.97,0.95,0.0,1.0,0.91
precision,0.95,0.97,0.94,0.0,1.0,1.00
recall,0.98,0.97,0.96,0.0,1.0,1.00
number,55.00,129.00,169.00,0.0,4.0,3.00


-------------------
Aggregated metrics (eval_gszh)
-------------------
Per-ent metrics (mean)

,PER,LOC,ORG,PERderiv,LOCderiv,ORGderiv
f1,0.91,0.93,0.92,0.0,0.87,0.69
precision,0.90,0.90,0.90,0.0,0.83,0.65
recall,0.93,0.96,0.94,0.0,0.95,0.74
number,39.20,106.20,246.40,0.0,10.40,14.80


Per-ent metrics (median)

,PER,LOC,ORG,PERderiv,LOCderiv,ORGderiv
f1,0.92,0.93,0.92,0.0,0.90,0.67
precision,0.91,0.88,0.89,0.0,0.92,0.65
recall,0.98,0.97,0.94,0.0,1.00,0.70
number,38.00,102.00,242.00,0.0,11.00,16.00


Some dense pandas code to get a heatmap table of all aggregated metrics (mean).

In [7]:
pd.concat(
    [t[1].drop("number") for t in mean_dfs], keys=[t[0] for t in mean_dfs]
).T.style.format("{:.2f}").background_gradient(
    cmap="RdYlGn", vmin=0.0, vmax=1.0
).set_properties(**{"font-size": "12px"})

The same for the mean frequency of named entity types per fold and dataset.

In [8]:
entity_freq_df = pd.concat(
    [t[1].drop(["F1", "P", "R"]) for t in mean_dfs], keys=[t[0] for t in mean_dfs]
).T.droplevel(1, axis="columns")
entity_freq_df.style.format("{:.2f}").background_gradient(
    cmap="RdYlGn", vmin=entity_freq_df.min().min(), vmax=entity_freq_df.max().max()
).set_properties(**{"font-size": "12px"})

,all,krp_19jhd,krp_20jhd,rrb_19jhd,rrb_20jhd,gszh
PER,487.20,55.80,284.00,51.20,57.00,39.20
LOC,704.80,30.60,358.00,78.40,131.60,106.20
ORG,1108.20,68.20,453.80,163.80,176.00,246.40
PERderiv,1.20,0.00,0.00,1.20,0.00,0.00
LOCderiv,38.40,1.40,15.80,6.80,4.00,10.40
ORGderiv,48.20,2.60,22.20,3.80,4.80,14.80
